In [2]:
import struct
import numpy as np
from torch.utils.data import Dataset

def load_idx_images(filename):
    with open(filename, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        data = np.frombuffer(f.read(), dtype=np.uint8)
        data = data.reshape(num, rows, cols)
        return data

def load_idx_labels(filename):
    with open(filename, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

class MNIST_IDX(Dataset):
    def __init__(self, img_file, lbl_file, transform=None):
        self.images = load_idx_images(img_file)
        self.labels = load_idx_labels(lbl_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = int(self.labels[idx])

        if self.transform:
            img = self.transform(img)

        return img, label

In [3]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch

transform = transforms.Compose([
    transforms.ToTensor(),
])

trainset = MNIST_IDX("train-images.idx3-ubyte", "train-labels.idx1-ubyte", transform)
testset  = MNIST_IDX("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte", transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = DataLoader(testset, batch_size=64, shuffle=False)


In [4]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),  # 28×28 → 28×28
            nn.ReLU(),
            nn.MaxPool2d(2),                 # 28 → 14

            nn.Conv2d(32, 64, 3, padding=1), # 14×14
            nn.ReLU(),
            nn.MaxPool2d(2)                  # 14 → 7
        )

        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


In [5]:
class RNN_MNIST(nn.Module):
    def __init__(self, hidden_size=128):
        super().__init__()
        self.lstm = nn.LSTM(input_size=28, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 10)

    def forward(self, x):
        # x shape: (batch, 1, 28, 28)
        x = x.squeeze(1)      # → (batch, 28, 28)
        out, _ = self.lstm(x)
        out = out[:, -1, :]   # last timestep
        return self.fc(out)


In [6]:
def train(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [7]:
def test(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            _, pred = torch.max(out, 1)

            total += y.size(0)
            correct += (pred == y).sum().item()

    return 100 * correct / total


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = nn.CrossEntropyLoss()

# --- Train CNN ---
cnn = CNN().to(device)
opt_cnn = torch.optim.Adam(cnn.parameters(), lr=0.001)

for epoch in range(3):
    loss = train(cnn, trainloader, opt_cnn, loss_fn, device)
    acc = test(cnn, testloader, device)
    print(f"CNN Epoch {epoch+1} | Loss={loss:.4f} | Acc={acc:.2f}%")

# --- Train RNN ---
rnn = RNN_MNIST().to(device)
opt_rnn = torch.optim.Adam(rnn.parameters(), lr=0.001)

for epoch in range(3):
    loss = train(rnn, trainloader, opt_rnn, loss_fn, device)
    acc = test(rnn, testloader, device)
    print(f"RNN Epoch {epoch+1} | Loss={loss:.4f} | Acc={acc:.2f}%")


CNN Epoch 1 | Loss=0.1772 | Acc=98.43%
CNN Epoch 2 | Loss=0.0510 | Acc=98.36%
CNN Epoch 3 | Loss=0.0344 | Acc=98.85%
RNN Epoch 1 | Loss=0.5051 | Acc=93.83%
RNN Epoch 2 | Loss=0.1461 | Acc=96.36%
RNN Epoch 3 | Loss=0.0966 | Acc=97.32%
